In [6]:
from getpass import getpass
import pymysql as sql
import random
import math
import smtplib


def db_connect():
    db = sql.connect(user='root', password='', host='localhost', port=3306, database='python1pm')
    cur = db.cursor()
    return db, cur

def passkey(password):
    lower = 0
    upper = 0
    digit = 0
    special = 0
    for i in password:
        if i.isdigit():
            digit += 1
        elif i.islower():
            lower += 1
        elif i.isupper():
            upper += 1
        elif not i.isidentifier():
            special += 1

    return lower >= 1 and upper >= 1 and digit >= 1 and special >= 1 and len(password) >= 8

def send_otp_email(receiver_email, otp):
    digits = "0123456789"
    OTP = ""
    for i in range(6):
        OTP += digits[math.floor(random.random() * 10)]
    msg = OTP + " is your OTP"
    
    try:
        s = smtplib.SMTP('smtp.gmail.com', 587)
        s.starttls()
        s.login("projectnsp2021@gmail.com", "pedmexjajuwkwvsb")
        s.sendmail("projectnsp2021@gmail.com", receiver_email, msg)
        s.quit()
        return OTP
    except Exception as e:
        print("Failed to send OTP email:", str(e))
        return None

def signup():
    try:
        username=input('Enter your name: ')
        password= getpass('Enter your password: ')

        if passkey(password):
            print(' Password is valid...')  

            print('Initial amount of 2000 is required to open an account: ')
            choice=input('Enter Your Choice(y/n): ').lower()
            if choice == 'y':
                # gen acc_no 
                db, cur =db_connect()
                cmd="select * from bank_data order by acc_no desc limit 1"
                cur.execute(cmd)
                data =cur.fetchall()
                new_acc=data[0][0]+1
                balance =2000
                cmd1=f"insert into bank_data values({new_acc}, '{username}', '{password}',{balance})"
                cur.execute(cmd1)
                db.commit()
                db.close()
                print('Your account is ctreated...')
                print(f'Your account number is {new_acc}')
            else:
                print('Initial amount is required. Thankyou for giving your time')

        else:
            print('Invalid Password. Try again...')
    except Exception as e:
        print(f"An error occurred")
            
def login(): 
    try:
        new_acc = int(input('Enter the account number: '))
        password = getpass('Enter the password: ')

        db, cur = db_connect()
        cmd3 = f"SELECT acc_no, password, balance FROM bank_data WHERE acc_no = {new_acc}"
        cur.execute(cmd3)
        data = cur.fetchone()

        if data and new_acc == data[0] and password == data[1]:
            while True:
                print("Login Successful:")
                print("1. Credit")
                print("2. Debit")
                print("3. Change Password")
                print("4. Check Balance")
                print("5. Show Details")
                print("6. Logout")

                choice = input("Enter your choice: ")

                if choice == '1':
                    amount = float(input("Enter the amount to credit: "))
                    if amount > 0:
                        new_balance = data[2] + amount

                        cmd_credit = f"UPDATE bank_data SET balance = {new_balance} WHERE acc_no = {new_acc}"
                        cur.execute(cmd_credit)
                        db.commit()
                        print(f"Amount {amount} credited successfully. New balance: {new_balance}")
                    else:
                        print("Invalid amount. Please enter a positive amount.")

                elif choice == '2':
                    amount = float(input("Enter the amount to debit: "))
                    if amount > 0 and amount <= data[2]:
                        new_balance = data[2] - amount

                        cmd_debit = f"UPDATE bank_data SET balance = {new_balance} WHERE acc_no = {new_acc}"
                        cur.execute(cmd_debit)
                        db.commit()
                        print(f"Amount {amount} debited successfully. New balance: {new_balance}")
                    else:
                        print("Invalid amount or insufficient balance.")

                elif choice == '3':
                    new_password = getpass("Enter the new password: ")
                    
                    if passkey(new_password):
                        # Generate and send OTP via email
                        receiver_email = input("Enter your email: ")
                        otp = send_otp_email(receiver_email)

                        if otp:
                            entered_otp = input("Enter the OTP received : ")

                            if entered_otp == otp:
                                cmd_reset_password = f"UPDATE bank_data SET password = '{new_password}' WHERE acc_no = {new_acc}"
                                cur.execute(cmd_reset_password)
                                db.commit()
                                db.close()
                                print("Password changed successfully.")
                            else:
                                print("Invalid OTP. Password reset failed.")
                        else:
                            print("Email OTP could not be sent. Password reset failed.")
                    else:
                        print("New password should contain at least one upper case, one lower case, one digit, and one special character, and be at least 8 characters long.")
                elif choice == '4':
                    print(f"Your Current Balance is: {data[2]}")

                elif choice == '5':
                    print(f"Your Bank Details are: Account Number - {data[0]}, Balance - {data[2]}")

                elif choice == '6':
                    print("Logging out...")
                    break

                else:
                    print("Invalid choice. Please choose a valid option.")
        else:
            print('Incorrect Account Number and Password. Please try again.')
    except Exception as e:
        print(f"An error occurred")

def reset_password():
    new_acc = int(input('Enter the account number: '))
    password = getpass('Enter the password: ')

    db, cur = db_connect()
    cmd4 = f"SELECT acc_no, password, balance FROM bank_data WHERE acc_no = {new_acc}"
    cur.execute(cmd4)
    data = cur.fetchone()

    if data and password != data[1]:
        print("Invalid Password")
        choice = input('Do you want to reset the Password (Yes/No): ')
        if choice.lower() == 'yes':
            new_password = getpass("Enter the new password: ")

            if passkey(new_password):
                # Generate and send OTP via email
                receiver_email = input("Enter your email: ")
                otp = send_otp_email(receiver_email)

                if otp:
                    entered_otp = input("Enter the OTP received: ")

                    if entered_otp == otp:
                        cmd_reset_password = f"UPDATE bank_data SET password = '{new_password}' WHERE acc_no = {new_acc}"
                        cur.execute(cmd_reset_password)
                        db.commit()
                        db.close()
                        print("Password changed successfully.")
                    else:
                        print("Invalid OTP. Password reset failed.")
                else:
                    print("Email OTP could not be sent. Password reset failed.")
            else:
                print("New password should contain at least one upper case, one lower case, one digit, and one special character, and be at least 8 characters long.")
        else:
            print("Login Failed!")
    else:
        print("Previous Password cannot be Used")

def exit_program():
    try:    
        print('Exit Successful!!')
    except Exception as e:
        print(f"An error occurred")

if __name__ == "__main__":
    while True:
        print("\nWelcome to the Banking Application:")
        print("1. Signup")
        print("2. Login")
        print("3. Forgot Password")
        print("4. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            signup()
        elif choice == '2':
            login()
        elif choice == '3':
            reset_password()
        elif choice == '4':
            exit_program()
            break
        else:
            print("Invalid choice. Please choose a valid option.")



Welcome to the Banking Application:
1. Signup
2. Login
3. Forgot Password
4. Exit
Enter your choice: 4
Exit Successful!!
